In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/com.github/lucifer1004/AdventOfCode.jl`


In [2]:
using AdventOfCode

In [3]:
year = 2022
day = 11

11

In [4]:
input = get_input(year, day)

"Monkey 0:\n  Starting items: 62, 92, 50, 63, 62, 93, 73, 50\n  Operation: new = old * 7\n  Test: divisible by 2\n    If true: throw to monkey 7\n    If false: throw to monkey 1\n\nMonkey 1:\n  Starting items: 51, 97, 74, 84, 99\n  Operation: new = old + 3\n  Test: divisible by 7\n " ⋯ 735 bytes ⋯ " 78\n  Operation: new = old + 8\n  Test: divisible by 3\n    If true: throw to monkey 0\n    If false: throw to monkey 7\n\nMonkey 7:\n  Starting items: 84, 93, 54\n  Operation: new = old + 1\n  Test: divisible by 17\n    If true: throw to monkey 2\n    If false: throw to monkey 1"

In [5]:
parsed_input = parse_input(input)

55-element Vector{SubString{String}}:
 "Monkey 0:"
 "  Starting items: 62, 92, 50, 63, 62, 93, 73, 50"
 "  Operation: new = old * 7"
 "  Test: divisible by 2"
 "    If true: throw to monkey 7"
 "    If false: throw to monkey 1"
 ""
 "Monkey 1:"
 "  Starting items: 51, 97, 74, 84, 99"
 "  Operation: new = old + 3"
 "  Test: divisible by 7"
 "    If true: throw to monkey 2"
 "    If false: throw to monkey 4"
 ⋮
 "  Starting items: 78"
 "  Operation: new = old + 8"
 "  Test: divisible by 3"
 "    If true: throw to monkey 0"
 "    If false: throw to monkey 7"
 ""
 "Monkey 7:"
 "  Starting items: 84, 93, 54"
 "  Operation: new = old + 1"
 "  Test: divisible by 17"
 "    If true: throw to monkey 2"
 "    If false: throw to monkey 1"

In [6]:
sample = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

"Monkey 0:\n  Starting items: 79, 98\n  Operation: new = old * 19\n  Test: divisible by 23\n    If true: throw to monkey 2\n    If false: throw to monkey 3\n\nMonkey 1:\n  Starting items: 54, 65, 75, 74\n  Operation: new = old + 6\n  Test: divisible by 19\n    If true: throw to monk" ⋯ 67 bytes ⋯ "9, 60, 97\n  Operation: new = old * old\n  Test: divisible by 13\n    If true: throw to monkey 1\n    If false: throw to monkey 3\n\nMonkey 3:\n  Starting items: 74\n  Operation: new = old + 3\n  Test: divisible by 17\n    If true: throw to monkey 0\n    If false: throw to monkey 1"

In [7]:
parsed_sample = parse_input(sample)

27-element Vector{SubString{String}}:
 "Monkey 0:"
 "  Starting items: 79, 98"
 "  Operation: new = old * 19"
 "  Test: divisible by 23"
 "    If true: throw to monkey 2"
 "    If false: throw to monkey 3"
 ""
 "Monkey 1:"
 "  Starting items: 54, 65, 75, 74"
 "  Operation: new = old + 6"
 "  Test: divisible by 19"
 "    If true: throw to monkey 2"
 "    If false: throw to monkey 0"
 ⋮
 "  Starting items: 79, 60, 97"
 "  Operation: new = old * old"
 "  Test: divisible by 13"
 "    If true: throw to monkey 1"
 "    If false: throw to monkey 3"
 ""
 "Monkey 3:"
 "  Starting items: 74"
 "  Operation: new = old + 3"
 "  Test: divisible by 17"
 "    If true: throw to monkey 0"
 "    If false: throw to monkey 1"

In [8]:
struct Monkey
    id::Int
    items::Vector{Int}
    fn::Expr
    test::Int
    if_true::Int
    if_false::Int
end

In [9]:
function Monkey(desc)
    id = parse(Int, match(r"Monkey (\d+):", desc[1])[1]) + 1
    starting_items = map(x -> parse(Int, x), split(match(r"Starting items: (.*)", desc[2])[1], ", "))
    fn = Meta.parse("function f$(id)(old) " * match(r"Operation: new = (.*)", desc[3])[1] * " end")
    test = parse(Int, match(r"Test: divisible by (.*)", desc[4])[1])
    if_true = parse(Int, match(r"If true: throw to monkey (\d+)", desc[5])[1]) + 1
    if_false = parse(Int, match(r"If false: throw to monkey (\d+)", desc[6])[1]) + 1

    return Monkey(id, starting_items, fn, test, if_true, if_false)
end

Monkey

In [10]:
function preprocess(input)
    i = 1
    monkeys = []
    while i + 5 <= length(input)
        push!(monkeys, Monkey(input[i:i+5]))
        i += 7
    end
    return monkeys
end

preprocess (generic function with 1 method)

In [11]:
macro part_one(input)
	quote
		monkeys = preprocess($input)
		n = length(monkeys)
		busy = zeros(Int, n)
		items = [collect(monkey.items) for monkey in monkeys]
		fns = [eval(monkey.fn) for monkey in monkeys]

		for rnd in 1:20
			for i in 1:n
				for item in items[i]
					busy[i] += 1
					nxt = fns[i](item) ÷ 3
					if nxt % monkeys[i].test == 0
						push!(items[monkeys[i].if_true], nxt)
					else
						push!(items[monkeys[i].if_false], nxt)
					end
				end
				empty!(items[i])
			end
		end

		@show busy
		sort!(busy)
		return busy[end] * busy[end - 1]
	end
end

@part_one (macro with 1 method)

In [12]:
@part_one(parsed_sample)

busy = [101, 95, 7, 105]


10605

In [13]:
part_one_ans = @eval @part_one(parsed_input)

busy = [291, 296, 17, 297, 306, 7, 6, 115]


90882

In [14]:
submit_answer(year, day, part_one_ans)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 11 - Advent of Code 2022</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3329 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"

In [15]:
macro part_two(input)
	quote
		monkeys = preprocess($input)
		n = length(monkeys)
		busy = zeros(Int, n)
		MOD = lcm([monkeys[i].test for i in 1:n])
		items = [collect(monkey.items) for monkey in monkeys]
		fns = [eval(monkey.fn) for monkey in monkeys]

		for rnd in 1:10000
			for i in 1:n
				for item in items[i]
					busy[i] += 1
					nxt = fns[i](item) % MOD
					if nxt % monkeys[i].test == 0
						push!(items[monkeys[i].if_true], nxt)
					else
						push!(items[monkeys[i].if_false], nxt)
					end
				end
				empty!(items[i])
			end
		end

		@show busy
		sort!(busy)
		return busy[end] * busy[end - 1]
	end
end

@part_two (macro with 1 method)

In [16]:
@part_two(parsed_sample)

busy = [52166, 47830, 1938, 52013]


2713310158

In [17]:
part_two_ans = @eval @part_two(parsed_input)

busy = [170248, 174243, 3549, 161508, 177299, 16284, 28995, 28223]


30893109657

In [18]:
submit_answer(year, day, part_two_ans, 2)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 11 - Advent of Code 2022</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3397 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"